In [1]:
import numpy as np
from scipy import stats
from scipy.stats import t

In [2]:
def z_critical(alpha, n_samples):
    t_crit = t.isf((alpha / (2 * n_samples)), (n_samples - 2))
    z_crit = ((n_samples - 1) / np.sqrt(n_samples)) * np.sqrt((t_crit ** 2) / (n_samples - 2 + (t_crit ** 2)))
    return z_crit

In [3]:
def delete_outliers(input_data, alpha = 0.05):
    """
    Replaces outliers in a 2d array with NaN, useful for colourmaps when outliers can make features invisible due to 
    scaling. The outliers are replaced with NaN because it is ignored when plotting colourmaps. Arguments: input_data is 
    a 2d array to remove outliers from, alpha is a significance level (0.05 is 95% confidence limit).
    """

    shape1 = input_data.shape[0]
    shape2 = input_data.shape[1]

    # Replace infinities with NaN
    fixed_data = input_data
    fixed_data[np.isinf(fixed_data)] = np.nan

    # Flatten, gets reshaped at the end
    fixed_data = fixed_data.flatten()

    outliers = []
    outliers_idxs = []
    z_scores = lambda data: abs(data - np.nanmean(data)) / np.nanstd(data)
    outlier_idx = lambda zs: np.nanargmax(zs)

    n_samples = len(fixed_data[np.isfinite(fixed_data)])
    grubbs_thresh = z_critical(alpha, n_samples)
    
    print(type(grubbs_thresh))
    
    zs = z_scores(fixed_data)
    outlier = np.nanmax(zs)
    
    #counter = 0
    #print(counter)
    
    while outlier > grubbs_thresh:
        
        # update the outliers
        idx = outlier_idx(zs)
        outliers.append(fixed_data[idx])
        outliers_idxs.append(idx)

        # Change outlier to nan
        fixed_data[idx] = np.nan

        # Find the new most extreme value
        zs = z_scores(fixed_data)
        outlier = np.nanmax(zs)

        # Find the new Grubbs threshold
        n_samples = len(fixed_data[np.isfinite(fixed_data)])
        grubbs_thresh = z_critical(alpha, n_samples)
        
        #counter += 1
        #print(counter)
    
    output_data = fixed_data.reshape(shape1, shape2)

    return output_data, outliers_idxs, outliers


In [5]:
# Testing
n = 100
test_data = np.array(np.random.rand(n)).reshape(10, 10)

test_data = test_data.flatten()

# Add in some inifnities to check it can handle them
test_data[np.random.randint(0, n, 2)] = -np.inf
test_data[np.random.randint(0, n, 2)] = np.inf

# Make some values huge
test_data[np.random.randint(0, n, 5)] = 1000000

# Return to original shape
test_data = test_data.reshape(10, 10)

processed = delete_outliers(test_data, 0.95)

print('Output array: ')
print(processed[0])
print('Outlier index: ')
print(processed[1])
print('Outliers: ')
print(processed[2])


<class 'numpy.float64'>
Output array: 
[[0.13212071        nan 0.94755023 0.89970571        nan 0.04830488
  0.277985   0.42294313 0.06650918 0.22387739]
 [0.88333609 0.2247593  0.81090556 0.48824332 0.68841422 0.90239296
  0.43929712 0.05911964        nan 0.10600593]
 [0.69117305 0.42418529 0.59085381 0.61312581 0.39960654 0.66118481
  0.55148808 0.16465778 0.18112308 0.33090194]
 [       nan 0.23949022 0.61318835 0.39174487 0.6531183  0.052388
  0.07280173        nan 0.01364209 0.06954415]
 [0.39957212 0.05567763 0.6579389  0.94771726 0.59994422 0.68423441
  0.31579547 0.91823265 0.2551594  0.28021967]
 [0.622558   0.69590443 0.5155312  0.20868139 0.08541889 0.07151166
  0.77012611        nan 0.37861779 0.52366507]
 [0.54588421 0.47936195 0.84705892 0.17969653        nan 0.1125763
  0.6956197  0.78109233 0.03937154 0.36587179]
 [0.83857881 0.60686192 0.44143435 0.08335895 0.5209275  0.17018344
  0.22158719 0.47225367 0.4128726  0.68559058]
 [       nan        nan 0.29824465 0.3390590